# AHRS

This Notebook showcases the most important classes and functions included in the Python package `ahrs`.

Here we will explore the basic use of:

- Class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html)
- Class [Quaternion](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternion.html)
- Class [QuaternionArray](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternionArray.html)
- The new class [Sensors](https://ahrs.readthedocs.io/en/latest/sensors.html) to simulate sensor data.
- The use of [Attitude estimation algorithms](https://ahrs.readthedocs.io/en/latest/filters.html).
- [Metrics functions](https://ahrs.readthedocs.io/en/latest/metrics.html) for orientation representations.
- The [World Magnetic Model](https://ahrs.readthedocs.io/en/latest/wmm.html)
- The [World Geodetic System](https://ahrs.readthedocs.io/en/latest/wgs84.html)
- And diverse tools included in `ahrs`.

### Helping Packages

Plotting and data-handling tools are imported from the script `tools.py` located in the current directory.

- `plot` shows time-series data in vertically stacked plots.
- `plot3` shows a 3D scene, where particles, frames, and items exist and interact in the same space.

Packages `matplotlib` and `ipympl` are required to build interactive visualizations in the Notebook. Make sure you have those installed.

These tools simplify the visualization of orientations in 3d, or time-series data, but are **NOT** included in the `ahrs` package.

Once you have `ahrs` installed (which also installs `numpy`) and you have the forementioned libraries, we can start by setting our notebook up.

In [ ]:
# Use widgets
%matplotlib widget

# Import NumPy
import numpy as np

# Seed random generator
GENERATOR = np.random.default_rng(42)

# Import plotting tools
from tools import plot
from tools import plot3

## AHRS Basics

We can start now. Let's import our favorite package.

In [ ]:
import ahrs

## Attitude Representations

The first elements we want to manipulate and use are the representations of any Attitude (that's the "A" in AHRS.)

There are many ways to mathematically represent the attitude:

1. Direction Cosine Matrix.
2. Euler Angles.
3. Axis-Angle.
4. Quaternion.

Let's check the most intuitive first.

### Direction Cosine Matrix

The [Direction Cosine Matrix](https://ahrs.readthedocs.io/en/latest/dcm.html)can be built with the class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html). This represents either:

- the **orientation** of a frame in 3D space with respect to another reference frame in the same space, or
- the **linear operation**, where a point (or points) is (are) rotated according the rotation described in the DCM.

Normally the global frame is represented by the 3x3 Identity matrix.

In [ ]:
global_frame = ahrs.DCM()    # An empty DCM is initialized as the global frame
print("Global frame:")
print(global_frame.view())
plot3(frames=global_frame)

As you can see, any frame in 3D can be visualized with three orthogonal axes of length one. The X-, Y-, and Z-axis are represented with red, green and blue color, respectively.

Each axis is described in the columns of the DCM:

$$
\begin{bmatrix}
  \color{red}{|} &  \color{green}{|} & \color{blue}{|} \\
  \color{red}{X} &  \color{green}{Y} & \color{blue}{Z} \\
  \color{red}{|} &  \color{green}{|} & \color{blue}{|}
 \end{bmatrix}
$$

To make it easier. Imagine:

- The first column describes the three-dimensional postition of the X-axis tip (red)
- The second column describes the three-dimensional postition of the Y-axis tip (green)
- The third column describes the three-dimensional postition of the Z-axis tip (blue)

Go back up and see that the visualized frame actually describes the columns of the global frame.

$$
\begin{array}{lcr}
X = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix} &
Y = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix} &
Z = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}
\end{array}
$$

The class `DCM` works in radians. If you want to use degrees, you can transform your values with the helper constant `DEG2RAD`.

Now let's built a matrix describing a rotation of 45° about the Z-axis, and confirm that its elements describe the position of each Axis' tip.

In [ ]:
rotated_frame = ahrs.DCM(z=45.0*ahrs.DEG2RAD)    # DCM rotated 45 degrees (see helper constant) about the Z-axis
print("Rotation of 45° about global frame's Z-axis:")
print(rotated_frame.view())
plot3(frames=rotated_frame)

You see the Z-axis (blue) remains unchanged, while the other points rotate around it, as expected.

The class `DCM` is derived from [NumPy arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html), and its attributes and methods are kept.

The DCM's characteristics in SO(3) are conveniently added. Operations between DCMs yield DCMs too.

You can also build the DCM, by giving it any valid $3\times 3$ orthogonal matrix in [SO(3)](https://en.wikipedia.org/wiki/3D_rotation_group).

In [ ]:
valid_rotation = ahrs.DCM(np.array([[np.sqrt(2)/2, -np.sqrt(2)/2, 0], [np.sqrt(2)/2, np.sqrt(2)/2, 0], [0, 0, 1]]))
valid_rotation.view()

It fails if it is not in SO(3)

In [ ]:
invalid_rotation = ahrs.DCM(np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

### Euler Angles

DCMs can be built in different ways, the most famous one is through Euler Angles.

These angles, first introduced by [Leonhard Euler](https://en.wikipedia.org/wiki/Leonhard_Euler) are three angles describing the orientation of an object with respect to a fixed coordinate system.

Three composed (chained) **elemental rotations** are always sufficient to reach any frame in 3D space. In this case, we can build each rotation separately with our class `DCM` by setting it in its constructor.

The chained multiplication of these orientations yields a final composed orientation.

Because the rotation operations are **always with respect to the initial global frame**, it is called an **[Extrinsic rotation](https://en.wikipedia.org/wiki/Euler_angles#Conventions_by_extrinsic_rotations)**.

In [ ]:
print("Rotation of 10 degrees about X-axis:")
print(ahrs.DCM(x=10.0*ahrs.DEG2RAD))

print("Rotation of 20 degrees about Y-axis:")
print(ahrs.DCM(y=20.0*ahrs.DEG2RAD))

print("Rotation of 30 degrees about Z-axis:")
print(ahrs.DCM(z=30.0*ahrs.DEG2RAD))

# New rotation matrix from products of rotations about X-, Y-, and Z-axis, respectively.
# Order of matrix multiplication is right to left: x --> y --> z
orientation = ahrs.DCM(z=30.0*ahrs.DEG2RAD) @ ahrs.DCM(y=20.0*ahrs.DEG2RAD) @ ahrs.DCM(x=10.0*ahrs.DEG2RAD)

print(f"Rotation Matrix {type(orientation)}:")
print(orientation)

This proces can be simplified at creation of the DCM object setting the [Euler angles](https://en.wikipedia.org/wiki/Euler_angles) tuple:

In [ ]:
orientation = ahrs.DCM( euler=('zyx', np.array([30.0, 20.0, 10.0])*ahrs.DEG2RAD) )
orientation.view()

As you can see, for the Euler angles we tried to match the mathematical order of multiplication from right to left.

The custom class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html) includes the basic attributes describing the mathematical properties of a DCM.

In [ ]:
attribute_list = ['T', 'I', 'inv', 'det', 'determinant', 'fro', 'frobenius', 'adj', 'adjugate', 'log']
for attribute_name in attribute_list:
    print(f"rotation.{attribute_name} =")
    print(orientation.__getattribute__(attribute_name), '\n')

For a full detail of its properties, see its [documentation](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html).

On top of that, it has a plethora of methods that help us to get extra information.

In [ ]:
# List all DCM methods and the first descriptive line from each docstring
from tools import describe_methods
describe_methods(orientation)

Have a look at the documentation of the class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html) for more details about the most important methods.

### Axis-Angle

A rotation can also be expressed with a pair parametrizing it. This pair is the axis of rotation as a three-dimensional vector, and its corresponding angle of rotation.

For example, for the rotation above, we simply indicate that we rotate 45° around the Z-axis like this:

In [ ]:
rotation = ahrs.DCM(axang=([0, 0, 1], 45*ahrs.DEG2RAD))
rotation.view()

Notice the axis **MUST** have a norm equal to 1.

$$
Z = \begin{bmatrix}0 \\ 0 \\ 1\end{bmatrix}
$$

To guarantee it the method [from_axisangle](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.from_axisangle.html) (used by the class DCM) normalizes it immediately.

In [ ]:
ahrs.DCM(axang=([0, 0, 2], 45*ahrs.DEG2RAD))

### Quaternions

The fourth and most useful representation explored here is the Quaternion.

With `ahrs` you build [unit Quaternions](https://en.wikipedia.org/wiki/Quaternion#Unit_quaternion), a.k.a. [Versors](https://en.wikipedia.org/wiki/Versor), even with an empty call, which returns the so-called `identity quaternion`:

$$
\mathbf{q} = \begin{bmatrix}1 & 0 & 0 & 0\end{bmatrix}
$$

In [ ]:
q = ahrs.Quaternion()
q.view()

As with the DCM, we can initialize our quaternions in different ways:

In [ ]:
print("Random quaternion:\n", ahrs.Quaternion(random=True))
print("4-element array:\n", ahrs.Quaternion([1., -2., 3., -4]))
print("3-element array:\n", ahrs.Quaternion([1., -2., 3.]))
print("From DCM:\n", ahrs.Quaternion(dcm=np.array([[np.sqrt(2)/2, -np.sqrt(2)/2, 0], [np.sqrt(2)/2, np.sqrt(2)/2, 0], [0, 0, 1]])))
print("Roll-pitch-yaw angles:\n", ahrs.Quaternion(rpy=np.array([30.0, 20.0, 10.0])*ahrs.DEG2RAD))

We can notice some things here:

- Quaternion inputs are ALWAYS normalized.
- When we cast the Quaternion as a [string](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str) (call it in a print) we get a "prettier" representation based on [Hamilton's initial description](https://en.wikipedia.org/wiki/History_of_quaternions#Hamilton's_discovery) of the basis vectors using $\mathcal{i}$, $\mathcal{j}$, and $\mathcal{k}$ for the vector part.
- When a quaternion is built with 3 items. It creates a pure quaternion, that is a quaternion with a scalar part equal to 0, while the 3 elements are set for the vector part.

Going back to the example above, we can represent the orientation as quaternion giving the values of the roll-pitch-yaw angles:

In [ ]:
q = ahrs.Quaternion(rpy=([0, 0, 45*ahrs.DEG2RAD]))
q.view()

This quaternion represents the same Direction Cosine Matrix of a 45° rotation around the Z-axis (yaw angle):

In [ ]:
q.to_DCM()

The class [Quaternion](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternion.html) is also subclassed from [NumPy Arrays](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html), and expands its attributes and methods describing the quaternion:

In [ ]:
describe_methods(q)

For a detailed description of each method and attribute, please check the class' [documentation](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternion.html)

Here are the available attributes:

In [ ]:
attribute_list = ['w', 'x', 'y', 'z', 'v', 'conjugate', 'conj', 'inverse', 'inv', 'exponential', 'exp', 'logarithm', 'log']
for attribute_name in attribute_list:
    print(f"{q.__class__.__name__}.{attribute_name} =")
    print(q.__getattribute__(attribute_name), '\n')

We cannot visualize the four-dimensional quaternion, but we can represent it in three-dimensions if we convert it to a DCM. Then, we can visualize it.

In [ ]:
dcm = q.to_DCM()
plot3(frames=dcm)

Quaternions are very versatile and, contrary to modern myths, very easy to use. For example, if we require the opposite. We merely negate the vector part. This "opposite" rotation/orientation is called the [quaternion conjugate](https://en.wikipedia.org/wiki/Quaternion#Conjugation,_the_norm,_and_reciprocal).

In [ ]:
q_c = q.conjugate
print(f"Quaternion conjugate:\n", q_c)
dcm_inv = ahrs.Quaternion(q_c).to_DCM()
plot3(frames=dcm_inv)

The original quaternion described a **counter-clockwise** rotation of 45° about the Z-axis, whereas its conjugate describes a **clockwise** rotation of 45° about the Z-axis.

### QuaternionArray

There is an extra class called [QuaternionArray](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternionArray.html), which, as the name suggests, handles more than one quaternion at once.

This simplifies repetitive operations between quaternions, and even gives computes new properties already.

Let's build an array of quaternions similar to the quaternion above, but different from each other by minor perturbances.

In [ ]:
Quaternions = ahrs.QuaternionArray(np.tile([1., 0., 0., np.sqrt(2)-1], (5, 1)) + np.random.randn(5, 4)*0.05)
Quaternions.view()

In this case, we can visualize the array of Quaternions using a simple time-series plot.

In [ ]:
plot(Quaternions)

We can see that every element is plotted per line, where the red line corresponds to the first column (scalar values, or $q_w$), green to the second column ($q_x$), blue to the third column ($q_y$), and gold to the fourth column ($q_z$)

In [ ]:
describe_methods(Quaternions)

Class `QuaternionArray` has almost all the same methods as the class `Quaternion`, but with some extra useful ones:

In [ ]:
# Mean Quaternion
Quaternions.average()

# WARNING: method `mean()` inherited from numpy.array
# will compute the numerical mean over all array values,
# which is NOT the Mean Quaternion

In [ ]:
# Angular Velocity around X-, Y-, and Z-axis from orientations as Quaternions (given sampling rate in seconds)
Quaternions.angular_velocities(0.1)  # Assumes sampling rate of 0.1 s (10 Hz)

In [ ]:
# Interpolate missing Quaternion using SLERP
Quaternions[2] = np.nan
print("Array missing 3rd quaternion:\n", Quaternions)

Quaternions.slerp_nan(inplace=True)
print("Array with SLERP:\n", Quaternions)

The quaternions $\begin{bmatrix}q_w & q_x & q_y & q_z\end{bmatrix}$ and $\begin{bmatrix}-q_w & -q_x & -q_y & -q_z\end{bmatrix}$ represent the same orientation, but some people prefer to have them more consistent.

The method [remove_jumps](https://ahrs.readthedocs.io/en/latest/quaternion/quaternionarray.remove_jumps.html) "flips" the opposite quaternions to make them more "pleasant" for the user.

In [ ]:
# Negate the 4th Quaternion (still same orientation)
Quaternions[3] *= -1
print("Array with negated quaternion (4th):\n", Quaternions)

# "Flip" quaternion back
Quaternions.remove_jumps()
print("Array with 4th quaternion 'flipped back':\n", Quaternions)

## Sensor Data

A great addition for the newest version is the class [Sensors](https://ahrs.readthedocs.io/en/latest/sensors.html), which simulates and synthesizes the IMU sensor signals from a given set of poses.

It assumes the poses come from a continuous recording, and it creates the corresponding sensor signals:

- Three-axial Gyroscope measurements given as rad/s.
- Three-axial Accelerometer measurements given as m/s^2.
- Three-axial Magnetometer measurements given as nT.

Giving the set of Quaternions from above we get:

In [ ]:
sensors = ahrs.Sensors(Quaternions)
print("Gyroscopes:\n", sensors.gyroscopes)
print("Accelerometers:\n", sensors.accelerometers)
print("Magnetometers:\n", sensors.magnetometers)
print(f"Sampling frequency: {sensors.frequency:.2f} Hz")
print(f"Number of samples: {sensors.num_samples:d}")

In [ ]:
plot(sensors.gyroscopes, sensors.accelerometers, sensors.magnetometers, ylabels=['rad/s', 'm/s^2', 'nT'])

## Attitude Estimators

Perhaps the most valued contribution of `ahrs` is its collection of attitude estimation algorithms. You can find a list [here](https://ahrs.readthedocs.io/en/latest/filters.html)

Let's jut explore one famous example: The [Madgwick Filter](https://ahrs.readthedocs.io/en/latest/filters/madgwick.html).

First, we generate some random orientations. When we build a `Sensors` object without giving it an array of orientations, it generates random orientations for us.

In [ ]:
simulated_data = ahrs.Sensors(num_samples=1000, in_degrees=False)
plot(simulated_data.quaternions,
     simulated_data.gyroscopes,
     simulated_data.accelerometers,
     simulated_data.magnetometers,
     ylabels = ['quaternions', 'rad/s', 'm/s^2', 'nT'])

Now that we generated IMU data, we can use it to estimate the original attitudes (orientations) with our Madgwick Filter.

In [ ]:
madgwick = ahrs.filters.Madgwick(gyr=simulated_data.gyroscopes,
                                 acc=simulated_data.accelerometers,
                                 mag=simulated_data.magnetometers,
                                 frequency=simulated_data.frequency)

Done!

The `Madgwick` object uses the given arrays to immediately perform the full computation of the orientations.

These orientations are in an $N\times 4$ array accessible in the attribute called `Q` (stands for Quaternions).

In [ ]:
plot(simulated_data.quaternions, madgwick.Q)

But what if we don't have the entire data beforehand?

We can initialize the `Madgwick` object without any sensor data, but _specifying an initial orientation_.

Once defined we estimate the orientations with any new data received, as we please.

In [ ]:
initial_quaternion = [1., 0., 0., 0.]
madgwick = ahrs.filters.Madgwick(q0=initial_quaternion)

In [ ]:
Quaternions = [initial_quaternion]    # Allocate first quaternion in an array
for index in range(1, simulated_data.num_samples):
    new_quaternion = madgwick.updateMARG(q=Quaternions[index-1],
                                         gyr=simulated_data.gyroscopes[index],
                                         acc=simulated_data.accelerometers[index],
                                         mag=simulated_data.magnetometers[index],
                                         dt=0.01)   # You can re-define the time-step (1/frequency) at each iteration
    Quaternions.append(new_quaternion)

This performs the exact operations as before, but we have more control of the computation at each estimation.

Now we have an array of N computed Quaternions. We can use our `Quaternion` class to handle them too.

In [ ]:
Quaternions = ahrs.QuaternionArray(Quaternions)
plot(Quaternions)

These operations are carried out similarly for many estimators included in `ahrs`.

If you want to use any other, please check its documentation first, to see how can you take advantage of them.

## Metrics

Finally, we can explore another useful set of tools included in `ahrs`: [Metrics for SO(3)](https://ahrs.readthedocs.io/en/latest/metrics.html).

These are a set of functions we can call to evaluate our estimated attitudes, or simply analyse them.

In [ ]:
from tools import describe_functions
describe_functions(ahrs.utils.metrics)

For example, let's compare the original quaternions generated by our simulator and the estimated ones from the Madgwick filter, by using the [Quaternion Angle Difference](https://ahrs.readthedocs.io/en/latest/metrics.html#ahrs.utils.metrics.qad):

In [ ]:
qad_differences = ahrs.utils.metrics.qad(simulated_data.quaternions, Quaternions)

This function returns one value between $0$ and $\pi$ per pair of Quaternions. A value of zero indicates the two compared quaternions are equal. The greater the value the more different they are.

In this case, it compared all original quaternions against all estimated quaternions. For this to work, both arrays must have the same amount of quaternions.

In [ ]:
plot(simulated_data.quaternions,
     Quaternions,
     qad_differences,
     ylabels=['original\nquaternions', 'estimated\nquaternions', 'QAD'])

We can clearly see the estimation slowly drifts away from the original pose.

This is an easy and quick way to estimate the accuracy of our estimations.

## World Models

Finally, an interesting tool included in the package is a set of [World Models](https://ahrs.readthedocs.io/en/latest/world_models.html).

These are functions that estimate the gravitational force and the magnetic field affecting an object.

### World Geodetic System

The [World Geodetic System 1984](https://ahrs.readthedocs.io/en/latest/wgs84.html) is the most common system used to estimate the gravitational force for any simple application.

It is based on a model, where a solar body (planet, moon, dwarf planet, etc.) has an ellipsoidal form. With a certain mass and other physical properties we could estimate the gravitaional pull at any latitude and height above this ellipsoidal body.

A more precise system is the [Earth Gravitational Model](https://en.wikipedia.org/wiki/Earth_Gravitational_Model) used by space agencies around the world, but this is not implemented in this package.

So, a simple construction of the class [WGS](https://ahrs.readthedocs.io/en/latest/wgs84.html#ahrs.utils.wgs84.WGS) will initialize our object:

In [ ]:
wgs = ahrs.utils.WGS()

a simple call to the method [normal_gravity](https://ahrs.readthedocs.io/en/latest/wgs84.html#ahrs.utils.wgs84.WGS.normal_gravity) giving the latitude and height above sea level computes the gravitational acceleration, in m/s^2, at that point.

In [ ]:
wgs.normal_gravity(50.0, 100.0)   # 50.0 degrees North, 100 m above sea level.

This WGS object has plenty of attributes describing the properties of the ellipsoidal body.

By default it builds the ellipsoidal model of the Earth.

In [ ]:
wgs_properties = [x for x in dir(wgs) if not (hasattr(wgs.__getattribute__(x), '__call__') or x.startswith('__'))]
max_column_width = len(max(wgs_properties, key=len))
for p in wgs_properties:
    print(f"{p:<{max_column_width}}  {wgs.__getattribute__(p)}")

4 main values define the entirety of the ellipsoid:

- The semi-major axis (labeled `a`) is the radius at the Equator of the ellipsoidal body.
- The flattenning (labeled `f`) is the "compression" of the ellipsoidal body. This can be used to define the semi-minor axis (labeled `b`) and many other properties.
- The [Standard Gravitational constant](https://en.wikipedia.org/wiki/Standard_gravitational_parameter) (labeled `GM`) is the product of the ellipsoid's [gravitational constant](https://en.wikipedia.org/wiki/Gravitational_constant) and its mass.
- The Ellipsoid's rotation rate (labeled `w`) is the rotation speed, in rad/s, of the ellipsoid.

For a different Ellipsoidal model, simply define these 4 properties and the WGS takes care of computing the rest.

A detailed description of all properties, their computation, and values, can be found in `ahrs`' [WGS documentation](https://ahrs.readthedocs.io/en/latest/wgs84.html#ahrs.utils.wgs84.WGS).

Other models included in the package are:

- The [International Gravity Formula](https://ahrs.readthedocs.io/en/latest/wgs84.html#ahrs.utils.wgs84.international_gravity). A simple equation calculating the normal gravity first defined in 1930. All its versions (1930, 1948, 1967, and 1980) are included too.
- The [WELMEC](https://ahrs.readthedocs.io/en/latest/wgs84.html#ahrs.utils.wgs84.welmec_gravity) gravity zone approximates the gravitational acceleration. This is mostly focused in European needs and conventions. USed mainly to calibrate measurement tools.

### WMM

The [World Magnetic Model](https://en.wikipedia.org/wiki/World_Magnetic_Model) describing Earth's magnetic field is also included in this package, and we can call it similarly using the class [WMM](https://ahrs.readthedocs.io/en/latest/wmm.html).

In [ ]:
wmm = ahrs.utils.WMM()

In this case, because Earth's magnetic field changes much faster than its gravitational forces, the [WMM](https://ahrs.readthedocs.io/en/latest/wmm.html) uses [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics) and a set of predefined coefficients to interpolate the magnetic field.

These coefficients are included in the COFF files of the package.

By default, WMM uses the values for the current day and the latitude, longitude and height above sea level of the city of Munich, where the creator of this package resides.

In [ ]:
wmm.magnetic_elements

Where there values are in nT and each represent:

| Element | Definition                                   |
|---------|----------------------------------------------|
| X       | Northerly intensity                          |
| Y       | Easterly intensity                           |
| Z       | Vertical intensity (Positive downwards)      |
| H       | Horizontal intensity                         |
| F       | Total intensity                              |
| I       | Inclination angle (a.k.a. dip angle)         |
| D       | Declination angle (a.k.a. magnetic variation)|
| GV      | Grivation (yearly magnetic "drift")          |

To retrieve the values at a any given point above Earth's surface, we have to give the desired date, latitude, longitude and height of that point above sea level.

In [ ]:
import datetime

# 34.14 degrees North, 118.35 degrees West, 500 m above sea level (WMM uses km), on the 26th October 2015
wmm = ahrs.utils.WMM(datetime.date(2015, 10, 26),
                     latitude=34.14160468409308,
                     longitude=-118.34978138071212,
                     height=0.5)
wmm.magnetic_elements

Each can be also accessed individually.

In [ ]:
wmm.X

In [ ]:
wmm.Y

In [ ]:
wmm.Z